<a href="https://www.kaggle.com/code/oumaimaaarabe/car-track?scriptVersionId=203704860" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

## load data

In [ ]:
# Imports
import os, warnings
import matplotlib.pyplot as plt
from matplotlib import gridspec

import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing import image_dataset_from_directory

warnings.filterwarnings("ignore")

def warn(*args, **kwargs):
    pass
warnings.warn = warn
# Reproducability
VALIDATION_SPLIT = 0.5  # Split test data 50-50 between validation and test
SEED = 123  # For reproducibility
BATCH_SIZE = 64
IMAGE_SIZE = [128, 128]

# Load training data (no split needed)
ds_train = image_dataset_from_directory(
    '../input/car-or-truck/train',
    labels='inferred',
    label_mode='binary',
    image_size=IMAGE_SIZE,
    interpolation='nearest',
    batch_size=BATCH_SIZE,
    shuffle=True,
)

# Split the test directory into validation and test sets
ds_val = image_dataset_from_directory(
    '../input/car-or-truck/valid',
    labels='inferred',
    label_mode='binary',
    image_size=IMAGE_SIZE,
    interpolation='nearest',
    batch_size=BATCH_SIZE,
    validation_split=VALIDATION_SPLIT,
    subset='training',  # This will be our validation set
    seed=SEED,
    shuffle=True,
)

ds_test = image_dataset_from_directory(
    '../input/car-or-truck/valid',  
    labels='inferred',
    label_mode='binary',
    image_size=IMAGE_SIZE,
    interpolation='nearest',
    batch_size=BATCH_SIZE,
    validation_split=VALIDATION_SPLIT,
    subset='validation',  # This will be our test set
    seed=SEED,
    shuffle=True,
)
# Data Pipeline
def convert_to_float(image, label):
    image = tf.image.convert_image_dtype(image, dtype=tf.float32)
    return image, label

AUTOTUNE = tf.data.experimental.AUTOTUNE
ds_train = (
    ds_train
    .map(convert_to_float)
    .cache()
    .prefetch(buffer_size=AUTOTUNE)
)
ds_valid = (
    ds_val
    .map(convert_to_float)
    .cache()
    .prefetch(buffer_size=AUTOTUNE)
)

ds_test = (
    ds_test
    .map(convert_to_float)
    .cache()
    .prefetch(buffer_size=AUTOTUNE)
)

## Define && Train Model

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers



# # detect and init the TPU
# tpu = tf.distribute.cluster_resolver.TPUClusterResolver()

# # instantiate a distribution strategy
# tf.tpu.experimental.initialize_tpu_system(tpu)
# tpu_strategy = tf.distribute.TPUStrategy(tpu)

# # instantiating the model in the strategy scope creates the model on the TPU
# with tpu_strategy.scope():



def create_model():
    model = keras.Sequential([

        # First Convolutional Block
        layers.Conv2D(filters=32, kernel_size=5, activation="relu", padding='same',
                      # [height, width, color channels(RGB)]
                      input_shape=[128, 128, 3]),
        layers.AveragePooling2D(2),

        # Second Convolutional Block
        layers.Conv2D(filters=64, kernel_size=3, activation="relu", padding='same'),
        layers.AveragePooling2D(2),

        # Third Convolutional Block
        layers.Conv2D(filters=128, kernel_size=3, activation="relu", padding='same'),
        layers.Conv2D(filters=128, kernel_size=3, activation="relu", padding='same'),
        layers.AveragePooling2D(2),

        # Classifier Head
        layers.Flatten(),
        layers.Dense(units=6, activation="relu"),
        layers.Dropout(0.2),
        layers.Dense(units=1, activation="sigmoid"),
    ])
    
    model.compile(
            optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
            loss='binary_crossentropy',
            metrics=[keras.metrics.BinaryAccuracy()],
    )
    return model


early_stopping = keras.callbacks.EarlyStopping(
    min_delta=0.001,
    patience=12,
    restore_best_weights=True,
    verbose=1)

ReduceLROnPlateau = keras.callbacks.ReduceLROnPlateau(
    monitor="val_loss",
    factor=0.1,
    patience=6,
    verbose=1,
    mode="min",
)


ModelCheckpoint = tf.keras.callbacks.ModelCheckpoint(
    "truckOrCar.keras",
    monitor='val_loss',
    verbose=1,
    save_best_only=True,
    mode='min',
)
model = create_model()
model.summary()

In [ ]:
# tf.config.optimizer.set_jit(False)
history = model.fit(
    ds_train,
    validation_data=ds_valid,
    epochs=50,
    verbose=1,
    callbacks=[early_stopping, ReduceLROnPlateau, ModelCheckpoint]
)

In [ ]:
import pandas as pd

history_frame = pd.DataFrame(history.history)
history_frame.loc[:, ['loss', 'val_loss']].plot()
history_frame.loc[:, ['binary_accuracy', 'val_binary_accuracy']].plot();

In [ ]:
import tensorflow as tf

# Replace 'your_image.jpg' with the actual filename of your uploaded image
img_path_c = '/kaggle/input/test-image/WHITE_cc_2022HOC020101_01_1280_BL.jpeg'
img_path = '/kaggle/input/truck-test/download_image.jpeg'

# Load the image using tf.keras.preprocessing.image.load_img
img = tf.keras.preprocessing.image.load_img(img_path_c, target_size=(128, 128))

# Convert the image to a NumPy array
img_array = tf.keras.preprocessing.image.img_to_array(img)

# Preprocess the image (e.g., normalize pixel values)
img_array = img_array / 255.0  # Normalize pixel values to [0, 1]

# Add an extra dimension to represent the batch size
img_array = tf.expand_dims(img_array, 0)

predictions = model.predict(img_array)
print(predictions)

In [ ]:
# model.save('car_or_truck_model')
loss, acc = model.evaluate(ds_test, verbose=2)
print("Untrained model, accuracy: {:5.2f}%".format(100 * acc))